In [17]:
import requests

main_subject = input("Enter your topic ").lower()

url = 'https://en.wikipedia.org/w/api.php'
params = {
        'action': 'query',
        'format': 'json',
        'generator':'links',
        'titles': main_subject,
        'prop':'pageprops',
        'ppprop':'wikibase_item',
        'gpllimit':1000,
        'redirects':1,
        }

r = requests.get(url, params=params)
r_json = r.json()
linked_pages = r_json['query']['pages']
page_titles = [p['title'] for p in linked_pages.values()]

In [18]:
page_titles[:5]

['AAAI Conference on Artificial Intelligence',
 'ACM Computing Classification System',
 'ACM Computing Surveys',
 'ADALINE',
 'AI boom']

In [19]:
num_articles = 100
pages = page_titles[:num_articles]

pages += [main_subject]

pages = list(set(pages))

In [20]:
import requests
from lxml import html
from tqdm.notebook import tqdm

text_db = []
for page in tqdm(pages):
    response = requests.get(
            'https://en.wikipedia.org/w/api.php',
            params={
                'action': 'parse',
                'page': page,
                'format': 'json',
                'prop':'text',
                'redirects':''
            }
        ).json()
        
    raw_html = response['parse']['text']['*']
    document = html.document_fromstring(raw_html)
    text = ''


    for p in document.xpath('//p'):
        text += p.text_content()
    text_db.append(text)
   
print('Done')

  0%|          | 0/101 [00:00<?, ?it/s]

Done


In [21]:
print('Number of articles extracted: ', len(text_db))

Number of articles extracted:  101


In [22]:
text_db[0][:1000]

'\nThe Association for Computational Linguistics (ACL) is a scientific and professional organization for people working on natural language processing.[1] Its namesake conference is one of the primary high impact conferences for natural language processing research, along with EMNLP.[2][3] The conference is held each summer in locations where significant computational linguistics research is carried out.\nIt was founded in 1962, originally named the Association for Machine Translation and Computational Linguistics (AMTCL). It became the ACL in 1968.[4] The ACL has a European (EACL),[5] a North American (NAACL),[6] and an Asian (AACL)[7] chapter.\nThe ACL was founded in 1962 as the Association for Machine Translation and Computational Linguistics (AMTCL). The initial membership was about 100. In 1965 the AMTCL took over the journal Mechanical Translation and Computational Linguistics. This journal was succeeded by many other journals: American Journal of Computational Linguistics (1974—

In [23]:
text_db[1][:500]

'Artificial Intelligence: A Modern Approach (AIMA) is a university textbook on artificial intelligence, written by Stuart J. Russell and Peter Norvig. It was first published in 1995 and the fourth edition of the book was released on 28 April 2020.[1] It is used in over 1400 universities worldwide[2] and has been called "the most popular artificial intelligence textbook in the world".[3] It is considered the standard text in the field of artificial intelligence.[4][5]\nThe book is intended for an u'

In [24]:
import pandas as pd
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

vec = TfidfVectorizer(
    stop_words=stop_words,
    ngram_range=(2,2), 
    use_idf=True
    )

tf_idf =  vec.fit_transform(text_db)

df = pd.DataFrame(
    tf_idf.toarray(),
    columns=vec.get_feature_names_out(),
    index=pages
    )


df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omiko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,00 00,00 100,00 consumers,00 however,000 00,000 000,000 10,000 11,000 17,000 195,...,σ2 index,σ3 index,σ4 hierarchy,σn contains,σn σ1,ḥisāb al,ẇ1 ẇ2,ẇ2 given,ἀριθμός arithmos,吳恩達 born
Association for Computational Linguistics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Artificial Intelligence: A Modern Approach,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bayesian optimization,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alan Mackworth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Artificial immune system,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
idf_df = pd.DataFrame(
    vec.idf_,
    index=vec.get_feature_names_out(),
    columns=['idf_weigths']
    )

idf_df.sort_values(by=['idf_weigths']).head(10)

,idf_weigths
machine learning,1.753772
artificial intelligence,1.818310
neural networks,2.159237
citation needed,2.223775
computer science,2.223775
neural network,2.257677
deep learning,2.329136
10 11,2.406097
12 13,2.446919
also known,2.446919


In [26]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize


from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


n_topics = 25

nmf = NMF(n_components=n_topics)

nmf_features = nmf.fit_transform(tf_idf)

norm_features = normalize(nmf_features)

c:\Python311\Lib\site-packages\sklearn\decomposition\_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [27]:
print('Original df: ', df.shape)
print('NMF Processed df: ', nmf.components_.shape)

Original df:  (101, 133406)
NMF Processed df:  (25, 133406)


In [28]:
components = pd.DataFrame(
    nmf.components_,
    columns=[df.columns]
    )

clusters = {}

for i in range(len(components)):
    clusters[i] = []
    loop = dict(components.loc[i,:].nlargest(25)).items()
    for k,v in loop:
        clusters[i].append({'q':k[0],'sim_score': v})

In [36]:
grouping = pd.DataFrame(clusters)
grouping['topic'] = grouping[0].apply(lambda x: x['q'])
grouping.drop(0, axis=1, inplace=True)
grouping.set_index('topic', inplace=True)

def show_queries(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x['q'])
    return df


clustered_queries = show_queries(grouping)
clustered_queries.head()

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
topic,,,,,,,,,,,,,,,,,,,,,
machine learning,activation functions,strong ai,immune system,computational biology,clustering algorithm,acm computing,artificial intelligence,auto gpt,mw parser,alphago zero,...,years ago,computably enumerable,citation needed,behavior analysis,automated decision,action selection,brain computer,basis functions,application security,recurrent neural
learning algorithms,activation function,chinese room,artificial intelligence,human genome,displaystyle cf,subject classification,conference artificial,large language,parser output,lee sedol,...,billion years,natural numbers,page needed,radical behaviorism,decision making,classical planning,computer interface,computational chemistry,web application,ctc trained
artificial intelligence,displaystyle w_,artificial intelligence,artificial immune,protein structures,data set,computing surveys,advancement artificial,language models,apache spark,alphago master,...,climate change,computability theory,data mining,operant conditioning,algorithmic transparency,planning problem,affective computing,linear combination,laboratories inc,trained lstm
anomaly detection,partial partial,ai researchers,immune systems,protein structure,hierarchical clustering,help wikipedia,association advancement,text image,output cs1,ke jie,...,livestock production,first order,chemical space,behavior therapy,algorithmic bias,planning problems,cognitive computing,ab initio,sbc laboratories,university edinburgh
deep learning,loss function,human level,field artificial,biological data,means clustering,stub help,intelligence aaai,gopher family,big data,fan hui,...,000 years,theorem proving,information retrieval,behavior analytic,decisions made,state variables,eeg based,vector space,southwestern bell,neural networks


In [37]:
data = {}

norm_df = pd.DataFrame(norm_features, index=pages)
for page in pages:
    
    recommendations = norm_df.loc[page,:]

   
    similarities = norm_df.dot(recommendations)

    data[page] = []
    loop = dict(similarities.nlargest(20)).items()
    for k, v in loop:
        if k != page:
            data[page].append({'q':k,'sim_score': v})

In [38]:
data

{'Association for Computational Linguistics': [{'q': 'BERT (language model)',
   'sim_score': 0.9998574629607397},
  {'q': 'Chatbot', 'sim_score': 0.9975135403452876},
  {'q': 'BLOOM (language model)', 'sim_score': 0.5292569617326478},
  {'q': 'Bank fraud', 'sim_score': 0.381137038065721},
  {'q': 'AI winter', 'sim_score': 0.3223724453034647},
  {'q': 'Cognitive computing', 'sim_score': 0.0955453234514093},
  {'q': 'Applications of artificial intelligence',
   'sim_score': 0.051980716504998754},
  {'q': 'Autoencoder', 'sim_score': 0.035332933076578386},
  {'q': 'Artificial neural network', 'sim_score': 0.032876504309445324},
  {'q': 'Caffe (software)', 'sim_score': 0.018927769589708238},
  {'q': 'Artificial intelligence in healthcare',
   'sim_score': 0.016901748417469723},
  {'q': 'Algorithm', 'sim_score': 0.01607881304586183},
  {'q': 'Alan Turing', 'sim_score': 0.013478450404104423},
  {'q': 'Attention (machine learning)', 'sim_score': 0.011318587213953063},
  {'q': 'Chinchilla AI',

In [40]:
recommender = pd.DataFrame(data)

def show_queries(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x['q'])
    return df

show_queries(recommender).head(10)

,Association for Computational Linguistics,Artificial Intelligence: A Modern Approach,Bayesian optimization,Alan Mackworth,Artificial immune system,Brain–computer interface,AT&T Labs,Bard (chatbot),Auto-GPT,Bioinformatics,...,AI safety,Automatic differentiation,BERT (language model),Approximate computing,AI boom,Agriculture,Artificial neural network,AlphaFold,Artificial general intelligence,Behaviorism
0,BERT (language model),Artificial immune system,Computational chemistry,AAAI Conference on Artificial Intelligence,Artificial Intelligence: A Modern Approach,Affective computing,Application security,AI boom,AI boom,Computational biology,...,AI takeover,ADALINE,Association for Computational Linguistics,Cluster analysis,Bard (chatbot),Biology,ADALINE,Computational biology,Chinese room,Approximate computing
1,Chatbot,AI safety,Basis function,Arthur Samuel (computer scientist),AI safety,Basic Books,Adaptive website,Anthropic,Bard (chatbot),AlphaFold,...,Chinese room,Backpropagation,Chatbot,CURE algorithm,Anthropic,Bank fraud,Backpropagation,Bioinformatics,AI takeover,Artificial intelligence systems integration
2,BLOOM (language model),ACM Computing Classification System,Chromosome (genetic algorithm),Artificial intelligence systems integration,Applications of artificial intelligence,Cognitive computing,Artificial intelligence systems integration,ChatGPT,Anthropic,Computational anatomy,...,Artificial general intelligence,Batch normalization,BLOOM (language model),BIRCH,ChatGPT,AI safety,Automatic differentiation,Computational anatomy,AI winter,Active learning (machine learning)
3,Bank fraud,ACM Computing Surveys,Active learning (machine learning),AI safety,Cognitive computing,ArXiv,ArXiv,Chinchilla AI,ChatGPT,Big data,...,Artificial intelligence,Activation function,Bank fraud,Anomaly detection,Chinchilla AI,Applications of artificial intelligence,Batch normalization,Big data,Artificial intelligence,Chromosome (genetic algorithm)
4,AI winter,AlexNet,Computational anatomy,Cognitive computing,Artificial intelligence in healthcare,Big data,Association rule learning,Auto-GPT,Chinchilla AI,ArXiv,...,AI winter,Artificial neuron,AI winter,Active learning (machine learning),Auto-GPT,Bayesian optimization,Activation function,ArXiv,AI safety,AI winter
5,Cognitive computing,ArXiv,Activation function,Artificial intelligence in healthcare,Automata theory,Artificial neural network,Bank fraud,BLOOM (language model),BLOOM (language model),Active learning (machine learning),...,Algorithmic bias,Artificial neural network,Cognitive computing,Association rule learning,BLOOM (language model),Alan Turing,Artificial neuron,Active learning (machine learning),Backdoor (computing),Alan Turing
6,Applications of artificial intelligence,BLOOM (language model),Applications of artificial intelligence,Andrew Ng,Adaptive website,Applications of artificial intelligence,AI winter,AI safety,AI safety,Caffe (software),...,Automated decision-making,AlexNet,Applications of artificial intelligence,Adversarial machine learning,AI safety,Artificial intelligence in healthcare,Christopher Bishop,Caffe (software),Alan Turing,Algorithm
7,Autoencoder,Cognitive computing,Adversarial machine learning,Applications of artificial intelligence,Chatbot,Canonical correlation,Caffe (software),Basic Books,Basic Books,Biology,...,Algorithmic transparency,Backdoor (computing),Artificial neural network,Automated machine learning,Basic Books,Algorithmic efficiency,Attention (machine learning),Artificial neural network,AlexNet,Autoencoder
8,Artificial neural network,AAAI Conference on Artificial Intelligence,machine learning,Artificial Intelligence: A Modern Approach,Algorithm,Artificial intelligence,Alan Turing,Artificial intelligence systems integration,Artificial intelligence systems integration,Artificial intelligence in healthcare,...,AAAI Conference on Artificial Intelligence,Analysis of algorithms,Autoencoder,machine learning,Artificial intelligence systems integration,Algorithmic bias,A